In [ ]:
import os
import pickle
import numpy as np
import tensorflow as tf
import time
import math
import skimage as ski
import skimage.io
import matplotlib.pyplot as plt

def evaluate(Y, Y_):
  pr = []
  Y_ = np.argmax(Y_,axis=1)
  n = max(Y_)+1
  M = np.bincount(n * Y_ + np.argmax(Y,axis=1), minlength=n*n).reshape(n, n)
  for i in range(n):
    tp_i = M[i,i]
    fn_i = np.sum(M[i,:]) - tp_i
    fp_i = np.sum(M[:,i]) - tp_i
    tn_i = np.sum(M) - fp_i - fn_i - tp_i
    recall_i = tp_i / (tp_i + fn_i)
    precision_i = tp_i / (tp_i + fp_i)
    pr.append( (recall_i, precision_i) )

  accuracy = np.trace(M)/np.sum(M)

  return accuracy

def shuffle_data(data_x, data_y):
  indices = np.arange(data_x.shape[0])
  np.random.shuffle(indices)
  shuffled_data_x = np.ascontiguousarray(data_x[indices])
  shuffled_data_y = np.ascontiguousarray(data_y[indices])
  return shuffled_data_x, shuffled_data_y

img_height = img_width = 32
num_channels = 3

#construct bag of words representation once the database is here
train_x,train_y = 

valid_size = 5000
n_values = np.max(train_y) + 1
train_y = np.eye(n_values)[train_y]
train_x, train_y = shuffle_data(train_x, train_y)
valid_x = train_x[:valid_size, ...]
valid_y = train_y[:valid_size, ...]
train_x = train_x[valid_size:, ...]
train_y = train_y[valid_size:, ...]

#normalisation not usefull for bag of words
#data_mean = train_x.mean((0,1,2))
#data_std = train_x.std((0,1,2))

#train_x = (train_x - data_mean) / data_std
#valid_x = (valid_x - data_mean) / data_std
#test_x = (test_x - data_mean) / data_std



def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x):
  return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

param_lambda=0.01

x = tf.placeholder(tf.float32, shape=[None, 32 , 32, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 10])


x_image = tf.reshape(x, [-1, 32, 32, 3])
#conv(16,5) -> relu() -> pool(3,2) -> conv(32,5) -> relu() -> pool(3,2)
# -> fc(256) -> relu() -> fc(128) -> relu() -> fc(10)

W_conv1 = weight_variable([5, 5, 3, 16])
b_conv1 = bias_variable([16])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1)
print (h_pool1.shape)

W_conv2 = weight_variable([5, 5, 16, 32])
b_conv2 = bias_variable([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2)

W_fc1 = weight_variable([8 * 8 * 32, 256])
b_fc1 = bias_variable([256])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8 * 8 * 32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([256, 128])
b_fc2 = bias_variable([128])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)


W_fc3 = weight_variable([128, 10])
b_fc3 = bias_variable([10])

y_conv = tf.matmul(h_fc2, W_fc3) + b_fc3

l2=sum([param_lambda*tf.nn.l2_loss(x) for x in [W_conv2,W_conv1,W_fc2,W_fc1,W_fc3]])
logits=y_conv
midloss=tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)
cross_entropy = tf.reduce_mean(midloss)+l2
loss=cross_entropy
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(1e-3, global_step=global_step, decay_steps=1, decay_rate=1-1e-3)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,global_step=global_step)
train_op=train_step
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

plot_data = {}
plot_data['train_loss'] = []
plot_data['valid_loss'] = []
plot_data['train_acc'] = []
plot_data['valid_acc'] = []
plot_data['lr'] = []

sess=tf.Session()
sess.run(tf.global_variables_initializer())
num_epochs=40
batch_size=250
num_batches=int(len(train_x)/batch_size)
num_batches_valid=int(len(valid_x)/batch_size)



for epoch_num in range(1, num_epochs + 1):
    wpr=sess.run([W_conv1])[0]
    #print (wpr.shape)
    draw_conv_filters(epoch_num,0,wpr,SAVE_DIR)
    train_x, train_y = shuffle_data(train_x, train_y)
    for step in range(num_batches):
        offset = step * batch_size
        # s ovim kodom pazite da je broj primjera djeljiv s batch_size
        batch_x = train_x[offset:(offset + batch_size), ...]
        batch_y = train_y[offset:(offset + batch_size)]
        feed_dict = {x: batch_x, y_: batch_y}
        start_time = time.time()
        run_ops = [train_op, loss, logits, learning_rate]
        ret_val = sess.run(run_ops, feed_dict=feed_dict)
        _, loss_val, logits_val, lr = ret_val
        duration = time.time() - start_time
        if (step+1) % 5 == 0:
            sec_per_batch = float(duration)
            format_str = 'epoch %d, step %d / %d, loss = %.2f (%.3f sec/batch)'
            print(format_str % (epoch_num, step+1, num_batches, loss_val, sec_per_batch))
    #print('Train error:')
    logits_train_l=[]
    loss_train_l=[]
    loss_train=0
    for step in range(num_batches):
        offset = step * batch_size
        # s ovim kodom pazite da je broj primjera djeljiv s batch_size
        batch_x = train_x[offset:(offset + batch_size), ...]
        batch_y = train_y[offset:(offset + batch_size)]
        feed_dict = {x: batch_x, y_: batch_y}
        run_ops = [midloss, logits]
        loss_train_v, logits_val = sess.run(run_ops, feed_dict=feed_dict)
        loss_train_l.append(loss_train_v)
        logits_train_l.append(logits_val)
    loss_train=np.mean(np.concatenate(tuple(loss_train_l),axis=0))
    #loss_train=0
    #for i in range(len(train_x)):
    #    loss_train+=sess.run([loss],feed_dict={x: train_x[i,np.newaxis], y_: train_y[i,np.newaxis]})[0]
    #loss_train/=len(train_x)
    logits_train=np.concatenate(tuple(logits_train_l),axis=0)
    train_acc = evaluate(logits_train, train_y)
    #print('Validation error:')
    logits_valid_l=[]
    loss_valid_l=[]
    loss_valid=0
    for step in range(num_batches_valid):
        offset = step * batch_size
        batch_x = valid_x[offset:(offset + batch_size), ...]
        batch_y = valid_y[offset:(offset + batch_size)]
        feed_dict = {x: batch_x, y_: batch_y}
        run_ops = [midloss, logits]
        loss_valid_v, logits_val = sess.run(run_ops, feed_dict=feed_dict)
        loss_valid_l.append(loss_valid_v)
        logits_valid_l.append(logits_val)
    loss_valid=np.mean(np.concatenate(tuple(loss_valid_l),axis=0))
    #loss_valid=0
    #for i in range(len(valid_x)):
    #    loss_train+=sess.run([loss],feed_dict={x: valid_x[i,np.newaxis], y_: valid_y[i,np.newaxis]})[0]
    #loss_valid/=len(valid_x)
    logits_valid=np.concatenate(tuple(logits_valid_l),axis=0)
    valid_acc = evaluate(logits_valid, valid_y)
    plot_data['train_loss'] += [loss_train]
    plot_data['valid_loss'] += [loss_valid]
    plot_data['train_acc'] += [train_acc]
    plot_data['valid_acc'] += [valid_acc]
    plot_data['lr'] += [lr]

loss_train=np.concatenate(tuple(loss_train_l),axis=0)
loss_valid=np.concatenate(tuple(loss_valid_l),axis=0)

max_loss=np.zeros((10))
max_loss_i=np.zeros((10))

for i in range(len(loss_train)):
    pom=loss_train[i]
    pom_i=np.argmin(max_loss)
    if pom>max_loss[pom_i]:
        max_loss[pom_i]=pom
        max_loss_i[pom_i]=i
max_loss_i=[int(x) for x in max_loss_i]
for i in max_loss_i:
    draw_image(train_x[i],data_mean,data_std,SAVE_DIR,i,logits_train[i],"train")

max_loss=np.zeros((10))
max_loss_i=np.zeros((10))
for i in range(len(loss_valid)):
    pom=loss_valid[i]
    pom_i=np.argmin(max_loss)
    if pom>max_loss[pom_i]:
        max_loss[pom_i]=pom
        max_loss_i[pom_i]=i
max_loss_i=[int(x) for x in max_loss_i]